In [ ]:
## AO* 
from queue import PriorityQueue
import random

# Define the goal state
GOAL_STATE = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 0]]


def generate_initial_state():
    """
    Generate a random initial state for the 8-puzzle problem.
    """
    numbers = list(range(9))
    random.shuffle(numbers)
    state = [numbers[i:i + 3] for i in range(0, 9, 3)]
    return state


def print_state(state):
    """
    Print the current state of the puzzle.
    """
    for row in state:
        print(" ".join(map(str, row)))
    print()


def misplaced_tiles(state):
    """
    Calculate the number of misplaced tiles heuristic for the given state.
    """
    count = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != GOAL_STATE[i][j]:
                count += 1
    return count


def generate_neighbors(state):
    """
    Generate neighboring states by swapping the blank space (0) with adjacent tiles.
    """
    neighbors = []
    row, col = find_blank_space(state)
    deltas = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Possible moves: right, left, down, up
    for dr, dc in deltas:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_state = [row[:] for row in state]  # Make a copy of the current state
            new_state[row][col], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[row][col]
            neighbors.append(new_state)
    return neighbors


def find_blank_space(state):
    """
    Find the row and column indices of the blank space (0).
    """
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j


def ao_star(initial_state):
    """
    AO* search algorithm to solve the 8-puzzle problem with the number of misplaced tiles heuristic.
    """
    open_set = PriorityQueue()
    open_set.put((misplaced_tiles(initial_state), 0, initial_state))
    came_from = {}
    g_score = {initial_state: 0}

    while not open_set.empty():
        _, _, current_state = open_set.get()

        if current_state == GOAL_STATE:
            path = [current_state]
            while current_state in came_from:
                current_state = came_from[current_state]
                path.append(current_state)
            path.reverse()
            return path

        for neighbor in generate_neighbors(current_state):
            tentative_g_score = g_score[current_state] + 1
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_state
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + misplaced_tiles(neighbor)
                open_set.put((f_score, tentative_g_score, neighbor))

    return None  # No solution found


if __name__ == "__main__":
    initial_state = generate_initial_state()
    print("Initial State:")
    print_state(initial_state)

    solution = ao_star(initial_state)

    if solution:
        print("\nSolution:")
        for i, state in enumerate(solution):
            print(f"Step {i + 1}:")
            print_state(state)
    else:
        print("No solution found.")
